In [2]:
import pandas as pd

In [3]:
doc_pd = pd.read_csv('../all.csv',sep="|")

In [4]:
doc_pd['rev1'][0]

'These are the perfect tights for my 5-year old. The tights are very well made and have already lasted several washings (hang dry). The color is beautiful, and my daughter loves that she can wear flip-flops to class like the big girls do.'

In [5]:
import re

def first_process(text):
    text = re.sub(r'[^\w\s\.]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)                                 # delete URL
    text = re.sub(r'  ',' ', text)                                                    # delete more space
    text = re.sub(r'([!()\-{};:,<>./?@#$%\^&*_~])\1+', lambda x: x.group()[0], text)  # delete more punc
    text = re.sub(r'[^\x00-\x7F]+',' ', text)                                         # replace non-ASCII with a space
    text = text.lower()
    text = text.strip()
    return text

In [6]:
column_rev = ['rev1','rev2','rev3','rev4','rev5','rev6','rev7','rev8']

In [7]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
data = []
for i in range(15):
  i = i + 45
  contain_cl = []
  for cl in column_rev:
    fp = first_process(doc_pd[cl][i])
    fp = re.split(r'(?<=[,.])(?<!\d.)\s', fp)
    contain_cl.append(fp)
  data.append(contain_cl)

In [38]:
column_abs = ['summ1','summ2','summ3']

In [74]:
doc_abs = []
doc_abs_one = []
for i in range(15):
  # i = i + 45
  contain_cl = []
  for cl in column_abs:
    sentcon = sent_tokenize(doc_pd[cl][i])
    sentcon = '\n'.join(sentcon)
    contain_cl.append(sentcon)

  doc_abs.append(contain_cl)
  doc_abs_one.append(doc_pd['summ1'][i])

In [40]:
doc_abs[0]

["This kitchen steamer is based on a classic design, works very well and is convenient to use.\nIt can steam a variety of foods, from vegetables and starches, to seafood and eggs.\nIt's rather large, which can be an issue if your counter space is limited.\nOverall, highly recommended.",
 'This steamer is an excellent product.\nIt does wonderful when cooking rice, boiling eggs, steaming vegetables and more!\nIt makes clean up a breeze and even has a handy screen for putting seasoning in what your cooking.\nDefinitley a nice addition to any kitchen!',
 'This is a very nice and roomy steamer.\nIt has plenty of space to steam multiple items or even just one.\nIt is on the larger side so it is hard to store when not in use.\nAll the food that is steamed in it comes out perfect and clean up is easy.\ni would recommend this to anyone.']

In [41]:
from nltk.tokenize import word_tokenize, sent_tokenize

djumlahKata = []
djumlahKalimat = []
for dt in data:
    for dtt in dt:
        tmp_dt = []
        for kata in dtt:
            tmp_dt.extend(kata.split(' '))
        djumlahKalimat.append(len(dtt))    
    djumlahKata.append(len(tmp_dt))

rjumlahKata = []
rjumlahKalimat = []
for dt_ringkasan in doc_abs:
    for index_dtr in dt_ringkasan:
        rjumlahKata.append(len(index_dtr.split(' ')))
        rjumlahKalimat.append(len(index_dtr.split('\n')))

In [42]:
import numpy as np
print(np.mean(rjumlahKalimat)), print(np.mean(rjumlahKata))

3.9555555555555557
53.0


(None, None)

In [71]:
print(np.mean(rjumlahKalimat)), print(np.mean(rjumlahKata))

6.088888888888889
244.86666666666667


(None, None)

In [43]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary-multilingual)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)

c:\Users\Admin\anaconda3\envs\ranggi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The pre-trained model you are loading is an uncased model but you have set `do_lower_case` to False. We are setting `do_lower_case=True` for you but you may want to check this behavior.


In [44]:
def sentence_emb(text, model):
  marked_text = "[CLS] " + text + " [SEP]"

  # Tokenize our sentence with the BERT tokenizer.
  tokenized_text = tokenizer.tokenize(marked_text)
  segments_ids = [1] * len(tokenized_text)

  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Put the model in "evaluation" mode, meaning feed-forward operation.
  model.eval()

  # Predict hidden states features for each layer
  with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor, segments_tensors)

  # Concatenate the tensors for all layers. We use `stack` here to
  # create a new dimension in the tensor.
  token_embeddings = torch.stack(encoded_layers, dim=0)

  # Remove dimension 1, the "batches".
  token_embeddings = torch.squeeze(token_embeddings, dim=1)

  # Swap dimensions 0 and 1.
  token_embeddings = token_embeddings.permute(1,0,2)

  token_embeddings.size()

  # `encoded_layers` has shape [12 x 1 x 23 x 768]

  # `token_vecs` is a tensor with shape [23 x 768]
  token_vecs = encoded_layers[11][0]

  # Calculate the average of all 23 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)

  return sentence_embedding

In [45]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
import string
exclude = set(string.punctuation)

def cleaning(data, stem=False, stopword=False):
  cleaned_data = []
  if stem == False and stopword== False:
    for yy in data:
        filtered_sentence = []
        word_tokens = yy.split()
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)

        cleaned_data.append(' '.join(filtered_sentence))
  else:
    for yy in data:
        filtered_sentence = []
        yy = ''.join(ch for ch in yy if ch not in exclude)
        word_tokens = yy.split()
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)

        cleaned_data.append(' '.join(filtered_sentence))

  return cleaned_data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
from sklearn.decomposition import PCA
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.cluster import DBSCAN

In [47]:
from sklearn.metrics import silhouette_score

def get_silhouette (model_c, embedding, k_max):
  silhouette = {}

  for k in range(2, k_max):
    if model_c =='Agglomerative':
      model = AgglomerativeClustering(n_clusters=k).fit(embedding)
    elif model_c =='KMeans':
      model = KMeans(n_clusters=k, n_init=10).fit(embedding)
    else:
      model = KMedoids(n_clusters=k, init='k-medoids++').fit(embedding)
    silhouette[k] = silhouette_score(embedding, model.labels_)

  max_value = max(silhouette.values())
  max_index = max(silhouette, key=silhouette.get)

  return max_index, max_value

In [48]:
from sklearn.metrics import davies_bouldin_score

def dbi (model_c, embedding, k_max):
  if model_c == 'Agglomerative':
    model = AgglomerativeClustering(k_max).fit(embedding)
  elif model_c =='KMeans':
    model = KMeans(k_max, n_init=10).fit(embedding)
  else:
    model = KMedoids(k_max, init='k-medoids++').fit(embedding)

  dbi = davies_bouldin_score(embedding, model.labels_)

  return dbi

In [49]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn import metrics

def elbow(model_c, embedding, k_max):
  if model_c=='Agglomerative':
    elb_visualizer = KElbowVisualizer(AgglomerativeClustering(), k=(2,k_max), timings=False).fit(embedding)
  elif model_c=='KMeans':
    elb_visualizer = KElbowVisualizer(KMeans(n_init=10), k=(2,k_max), timings=False).fit(embedding)
  else:
    elb_visualizer = KElbowVisualizer(KMedoids(init='k-medoids++'), k=(2,k_max), timings=False).fit(embedding)

  max_index = elb_visualizer.elbow_value_
  max_value = elb_visualizer.elbow_score_

  return max_index, max_value

In [50]:
from sklearn.metrics import calinski_harabasz_score

def chi(model, embedding, k_max):
  if model=='Agglomerative':
    model = AgglomerativeClustering(k_max).fit(embedding)
  elif model=='KMeans':
    model = KMeans(n_clusters=k_max, n_init=10).fit(embedding)
  else:
    model = KMedoids(k_max, init='k-medoids++').fit(embedding)

  chi = calinski_harabasz_score(embedding, model.labels_)

  return chi

In [51]:
def n_clustering(model_c, embeddingr, k_max):
    pca = PCA(n_components=2)
    embedding = pca.fit_transform(embeddingr)
    clus = []

    ss, score_s = get_silhouette(model_c, embedding, k_max)
    el, score_e = elbow(model_c, embedding, k_max)

    clus = [ss, el]

    for cl in clus:
        if cl is None:
            clus.remove(cl)

    print(clus)
    clus.sort(reverse=True)
    clus_opt = {x:clus.count(x) for x in clus}
    index_max = max(clus_opt, key=clus_opt.get)

    score_chi = chi(model_c, embedding, index_max)
    score_dbi = dbi(model_c, embedding, index_max)
    alscore = [score_s, score_e, score_chi, score_dbi]
    print(index_max)

    return index_max, alscore

def clustering(model_c, embedding,n_cluster):
    if model_c =='Agglomerative':
      clus = AgglomerativeClustering(n_clusters=n_cluster).fit_predict(embedding)
    elif model_c =='KMeans':
      clus = KMeans(n_clusters=n_cluster, n_init=10).fit_predict(embedding)
    else:
      clus = KMedoids(n_clusters=n_cluster, init='k-medoids++').fit_predict(embedding)

    return clus

In [52]:
from collections import defaultdict

def clustering_sent(model_c, data, data_raw):
    data_ext = {}

    model = BertModel.from_pretrained('bert-base-uncased')

    for d in range(len(data)):

        embedding = np.empty((len(data[d]), model.config.hidden_size))


        for e in tqdm(range (len(data[d]))):
          embedding[e,:] = sentence_emb(data[d][e], model)

        tmp_txt = []
        tmp_clus = []
        k_max = 10

        print('data '+str(d))
        cls, score = n_clustering(model_c, embedding, k_max)
        agg = clustering(model_c, embedding, cls)

        for t in range(len(data_raw[d])):
            tmp_txt.append(data_raw[d][t])
            tmp_clus.append(agg[t])

        print(score)
        data_ext[d] = {}
        data_ext[d]['text'] = tmp_txt
        data_ext[d]['cluster'] = tmp_clus
        data_ext[d]['score_clus'] = score

    return data_ext

In [53]:
from functools import reduce

def flatten_clustering (data):
  flat_data = []
  for list_data in data:
    fd = []

    for yy in list_data:
      filtered_sentence = []
      yy = ''.join(ch for ch in yy if ch not in exclude)
      word_tokens = yy.split()
      for w in word_tokens:
          if w not in stop_words:
              filtered_sentence.append(w)

      fd.append(' '.join(filtered_sentence))

    flat_data.append(fd)

  return flat_data

In [54]:
data_1 = flatten_clustering(data)

In [55]:
data_1[2]

['weve used filters dustbuster several years.with little care occasionally blowing air tapping sidewalk long life.seem catch everything grit dog hair fine ashes without letting much blow exhaust.getting harder find local stores bought two.',
 'wrong filter buying chv1510 hand vacuum.amazon pairs together says works cyclonic hand vacs doesnt.you need vf110 chv1510. amazon wrong.product description wrong.me wrong reading reviews first.',
 'instead buy new dustbuster mine older filters arent stock anywhere didnt mean updrage pay newer one.after found filters buy work extremely well.very satisfied.',
 'bought replace tired dirty filter.after received realized nothing wrong one already had.they washable.however helps two change filter quickly necessary take time wash first one.',
 'item list listed vacuum frequently bought together black decker chv9608 9.6 volt cyclonicaction cordless dustbuster filter fit argh corrections needed frequently bought together section correct filter needs liste

In [56]:
model_c = 'K-Medoids++'
data_ext = clustering_sent(model_c, data_1, data)

NameError: name 'tqdm' is not defined

In [59]:
data_ext = pd.read_json("./clustering_amazon_kmedoids_testing.json").T

In [119]:
tmpde = []
for de in range (len(data_ext['text'][3])):
    if data_ext['cluster'][3][de] == 7:
        tmpde.append(data_ext['text'][3][de])
tmpde

[]

In [109]:
doc_abs[3]

["What an impressive Thomas the Train costume!\nIt will fit any train loving toddler for at least a few Halloween seasons due to its roomy design.\nThe candy pocket on the front is an adorable touch.\nAnd it's so stinkin' cute!\nAbsolutely worth the price.",
 'This Thomas costume is really cute plus it is comfortable and fits well.\nMy child liked that it has a pocket and a hat.\nIt even has a candy catcher.\nThis costume fits well for a two or three year old.\nThe hat is thin but it is completes the costume.\nMy child loved being Thomas.',
 'This Thomas the Tank Engine Halloween costume looks very cute, and fits well for children around two to five years old.\nIt comes with a large, built-in pouch for hands-free candy hauling.\nThe only potential issue is the fabric seems a bit thin and flimsy.\nNonetheless, I would recommend this.']

In [306]:
with open("./result_amazon/clustering_amazon_kmedoids_testing.json","w") as f:
    json.dump(data_ext,f, default=int)

In [42]:
import numpy as np

def cosine(u, v):
  return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [43]:
def content_relevance(centroid,hypothesis_embeddings):
  d = {}
  for sentence,embed in hypothesis_embeddings.items():
    d[sentence] = cosine(centroid,embed)

  return d

In [57]:
def sentence_novelty(content_relevance_score,hypothesis_embeddings):
  novel_sentences = {}
  TAU = 0.95
  for sent1,embed1 in hypothesis_embeddings.items():
    max_similarity = 0
    for sent2,embed2 in hypothesis_embeddings.items():
      if sent1!=sent2 and cosine(embed1,embed2)>max_similarity:
          max_similarity = cosine(embed1,embed2)

    if max_similarity<TAU:
      novel_sentences[sent1] = 1

    elif max_similarity>TAU and content_relevance_score[sent1]>content_relevance_score[sent2]:
      novel_sentences[sent1] = 1

    else:
      novel_sentences[sent1] = 1-max_similarity

  return novel_sentences

In [45]:
import math

def sentence_position(hypothesis):
  score_sent = {}
  for i,sent in enumerate(hypothesis):
      score_sent[sent] = max(0.5,math.exp(-(i+1)/(len(hypothesis)**(1/3))))

  return score_sent

In [46]:
import yake

def keyword_extractor(text):
    kword = []
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(text)

    for kw in range(len(keywords)):
        kword.append(keywords[kw])

    keywords = sorted(kword,key=lambda x:(-x[1],x[0]))

    return keywords

In [47]:
data_key = {}
for i in range(len(data)):
  tmp_dk = []
  # for dk in range(len(data_k[i])):
  tmp_dk = keyword_extractor(' '.join(data[i]))
  data_key[i] = tmp_dk

In [190]:
with open("./result_amazon/keyword_amazon.json","w") as f:
    json.dump(data_key,f, default=int)

In [49]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

vader_sentiment = SentimentIntensityAnalyzer()
def vader_sentiment_scores(text):
  score = vader_sentiment.polarity_scores(text)
  return score['compound']

def prediksiSentiment(kalimat, keyword):
    wordlist = len(word_tokenize(kalimat))
    val = abs(vader_sentiment_scores(kalimat)/wordlist)

    nk = []
    for k in range(len(keyword)):
      if keyword[k][0] in kalimat:
        nk.append(keyword[k][1])


    return val, np.sum(nk)

In [50]:
def normalization_sen(text, keyword):
  lprob = []
  for i,sent in enumerate(text):
    val, key = prediksiSentiment(sent, keyword)
    prob = key + val
    lprob.append(prob)

  score_sent = {}
  for i,sent in enumerate(text):
    score_sent[sent] = lprob[i]

  return score_sent

In [115]:
len(rjumlahKata), len(rjumlahKalimat)

(180, 180)

In [220]:
print(np.min(rjumlahKata)), print(np.mean(rjumlahKalimat))

135
5.988888888888889


(None, None)

CLUS + SENTIMENT + KEYWORD + POSITION

In [258]:
def summary2(text):
    extractive = []
    for dc,v in text.items():
        tmp_ext = []
        val_cls = [v for v in text[dc][0].values()]
        key_cls = [k for k in text[dc][0].keys()]
        for vc in range(len(val_cls)):
          if len(tmp_ext) < 10:
            tmp_ext.append(key_cls[vc])

        extractive.append('\n'.join(tmp_ext))
    return extractive

In [264]:
import rouge
evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l'],
                           max_n=4,
                           limit_length=False,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=True,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

CLUSTERING + RKiPS (Relevansi, Keterbaruan informasi, Posisi, Sentiment-keyword)

In [269]:
from tqdm import tqdm
from collections import defaultdict
def total_score(alpha, beta, gamma, teta, content_relevance_score, sentence_novelty_score, sentiment, sentence_pos):
  final_score = {}
  for sent in content_relevance_score:
    final_score[sent] = (alpha*content_relevance_score[sent])+(beta*sentence_novelty_score[sent])+(gamma*sentence_pos[sent])+(teta*sentiment[sent])
  final_score = {k: v for k, v in sorted(final_score.items(), key=lambda item: item[1],reverse=True)}
      
  return final_score

def convert_to_embeddings(alpha, beta, gamma, teta, text_sentences, text_embeddings, keyword, sentence_pos):
  hypothesis_embeddings = {}
  sum = 0
  index = 0

  for sent in text_sentences:
    var = text_embeddings[index]
    sum += var
    hypothesis_embeddings[sent] = var
    index += 1

  centroid = sum/len(text_sentences)
  
  content_relevance_score = content_relevance(centroid,hypothesis_embeddings)
  sentence_novelty_score = sentence_novelty(content_relevance_score,hypothesis_embeddings)
  phrasa_score = normalization_sen(text_sentences, keyword)
  # position_score = sentence_position(text_sentences)
  return total_score(alpha, beta, gamma, teta, content_relevance_score, sentence_novelty_score, phrasa_score, sentence_pos)

def iter_ext(data, data_pos, alpha, beta, gamma, teta):
    extractive = []
    index = 0
    
    result_dict = defaultdict(list)

    model = BertModel.from_pretrained('bert-base-uncased')

    for index in tqdm(range (len(data))):
        total = 0
        after_clus = []

        datapoint_length = {x:data[index]['cluster'].count(x) for x in data[index]['cluster']}
        sentence_pos = sentence_position(data_pos[index])
        for c in range(len(datapoint_length)):
            txt = []

            for t in range (len(data[index]['text'])):
                if data[index]['cluster'][t] == c:
                    txt.append(data[index]['text'][t])
        
            if len(txt) > 2:
               after_clus.extend(txt)
               
        embedding = np.empty((len(after_clus), model.config.hidden_size))

        for e in range (len(after_clus)):
          embedding[e,:] = sentence_emb(after_clus[e], model)

        predicted_summary = convert_to_embeddings(alpha, beta, gamma, teta, after_clus, embedding, data_key[index], sentence_pos) 
        result_dict[index].append(predicted_summary)

    return result_dict

# extract_skp = iter_ext(data_ext, data, 0.6, 0.2, 0.1, 0.1)
params = {'alpha': [0.8, 0.7, 0.7, 0.7, 0.7, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4, 0.3],
         'beta': [0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.3, 0.4, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.3, 0.3],
         'gamma': [0.05, 0.1, 0.05, 0.15, 0.05, 0.15, 0.10, 0.20, 0.10, 0.15, 0.05, 0.05, 0.05, 0.05, 0.15, 0.15, 0.2, 0.1, 0.2, 0.1, 0.2, 0.1, 0.2],
         'teta': [0.05, 0.1, 0.15, 0.05, 0.05, 0.15, 0.20, 0.10, 0.10, 0.05, 0.15, 0.05, 0.05, 0.15, 0.05, 0.15, 0.1, 0.2, 0.2, 0.1, 0.1, 0.2, 0.2],}
scores = []
for i in tqdm(range(len(params['alpha']))):
   extract_skp = iter_ext(data_ext, data, params['alpha'][i], params['beta'][i], params['gamma'][i], params['teta'][i])
   raw_ext1 = summary2(extract_skp)
   scores.append(evaluator.get_scores(raw_ext1, doc_abs))

100%|██████████| 23/23 [24:48<00:00, 64.72s/it]


In [295]:
import json
with open("./result_amazon/score_bobot_amazon2.json","w") as f:
    json.dump(scores,f, default=int)

RKiPS (Relevansi, Keterbaruan informasi, Posisi, Sentiment-keyword)

In [304]:
from tqdm import tqdm
from collections import defaultdict
def total_score(alpha, beta, gamma, teta, content_relevance_score, sentence_novelty_score, sentiment, sentence_pos):
  final_score = {}
  for sent in content_relevance_score:
    final_score[sent] = (alpha*content_relevance_score[sent])+(beta*sentence_novelty_score[sent])+(gamma*sentence_pos[sent])+(teta*sentiment[sent])
  final_score = {k: v for k, v in sorted(final_score.items(), key=lambda item: item[1],reverse=True)}
      
  return final_score

def convert_to_embeddings(alpha, beta, gamma, teta, text_sentences, text_embeddings, keyword, sentence_pos):
  hypothesis_embeddings = {}
  sum = 0
  index = 0

  for sent in text_sentences:
    var = text_embeddings[index]
    sum += var
    hypothesis_embeddings[sent] = var
    index += 1

  centroid = sum/len(text_sentences)
  
  content_relevance_score = content_relevance(centroid,hypothesis_embeddings)
  sentence_novelty_score = sentence_novelty(content_relevance_score,hypothesis_embeddings)
  phrasa_score = normalization_sen(text_sentences, keyword)
  # position_score = sentence_position(text_sentences)
  return total_score(alpha, beta, gamma, teta, content_relevance_score, sentence_novelty_score, phrasa_score, sentence_pos)

def iter_ext(data, data_pos, alpha, beta, gamma, teta):
    extractive = []
    index = 0
    
    result_dict = defaultdict(list)

    model = BertModel.from_pretrained('bert-base-uncased')

    for index in tqdm(range (len(data))):
        total = 0
        after_clus = []

        datapoint_length = {x:data[index]['cluster'].count(x) for x in data[index]['cluster']}
        sentence_pos = sentence_position(data_pos[index])
        for c in range(len(datapoint_length)):
            txt = []

            for t in range (len(data[index]['text'])):
                if data[index]['cluster'][t] == c:
                    txt.append(data[index]['text'][t])
        
            if len(txt) > 2:
               after_clus.extend(txt)
               
        embedding = np.empty((len(after_clus), model.config.hidden_size))

        for e in range (len(after_clus)):
          embedding[e,:] = sentence_emb(after_clus[e], model)

        predicted_summary = convert_to_embeddings(alpha, beta, gamma, teta, after_clus, embedding, data_key[index], sentence_pos) 
        result_dict[index].append(predicted_summary)

    return result_dict

In [299]:
extract_skp = iter_ext(data_ext, data, 0.5, 0.2, 0.2, 0.1)
raw_ext_noclus = summary2(extract_skp)
scores = evaluator.get_scores(raw_ext_noclus, doc_abs)
scores

100%|██████████| 60/60 [01:37<00:00,  1.63s/it]


{'rouge-1': {'f': 0.2690117798394361,
  'p': 0.1752289482179716,
  'r': 0.6010520842851004},
 'rouge-4': {'f': 0.00641273224077943,
  'p': 0.004156897201969006,
  'r': 0.014654083654294159},
 'rouge-3': {'f': 0.01829859985210641,
  'p': 0.011865531309560697,
  'r': 0.04164747130666827},
 'rouge-2': {'f': 0.06250133329902324,
  'p': 0.04053723678789259,
  'r': 0.14192282428051078},
 'rouge-l': {'f': 0.31320795603336704,
  'p': 0.21395400244019133,
  'r': 0.5992311156295129}}

In [314]:
extract_skp = iter_ext(data_ext, data, 0.5, 0.2, 0.2, 0.1)
raw_ext1 = summary2(extract_skp)
scores = evaluator.get_scores(raw_ext1, doc_abs)
scores

100%|██████████| 60/60 [01:26<00:00,  1.45s/it]


{'rouge-1': {'f': 0.26949531119454856,
  'p': 0.1761143593799731,
  'r': 0.5981582922091543},
 'rouge-4': {'f': 0.006335260248235896,
  'p': 0.004130400440106285,
  'r': 0.014245954820845948},
 'rouge-3': {'f': 0.018171933447394796,
  'p': 0.0118389101059539,
  'r': 0.040817879363812724},
 'rouge-2': {'f': 0.06250790885437686,
  'p': 0.040695351189257094,
  'r': 0.14062184494479985},
 'rouge-l': {'f': 0.3141889138149954,
  'p': 0.21523411020075872,
  'r': 0.5980659519836276}}

In [297]:
raw_ext2 = summary2(extract_p)
scores2 = evaluator.get_scores(raw_ext2, doc_abs)
scores2

{'rouge-1': {'f': 0.2664718876837354,
  'p': 0.1721553696830658,
  'r': 0.6072197073797587},
 'rouge-4': {'f': 0.005287931191420184,
  'p': 0.003365822762014105,
  'r': 0.012811296435416595},
 'rouge-3': {'f': 0.016397318777020376,
  'p': 0.010508635620930334,
  'r': 0.03858974023623702},
 'rouge-2': {'f': 0.059902274876962446,
  'p': 0.03853594864829872,
  'r': 0.1387032080072694},
 'rouge-l': {'f': 0.311098621032506,
  'p': 0.21093702258544758,
  'r': 0.6043437289360344}}

In [317]:
import json
with open("./rkips_data_amazon.json","w") as f:
    json.dump(extract_skp,f, default=int)

In [318]:
import json
with open("./rkip_data_amazon.json","w") as f:
    json.dump(extract_p,f, default=int)

In [232]:
from tqdm import tqdm
from collections import defaultdict
def total_score(alpha, beta, gamma, teta, content_relevance_score, sentence_novelty_score, sentiment, sentence_pos):
  final_score = {}
  for sent in content_relevance_score:
    final_score[sent] = (alpha*content_relevance_score[sent])+(beta*sentence_novelty_score[sent])+(gamma*sentiment[sent])+(teta*sentence_pos[sent])
  final_score = {k: v for k, v in sorted(final_score.items(), key=lambda item: item[1],reverse=True)}

  return final_score

def convert_to_embeddings(alpha, beta, gamma, teta, text_sentences, text_embeddings, keyword, sentence_pos):
  hypothesis_embeddings = {}
  sum = 0
  index = 0

  for sent in text_sentences:
    var = text_embeddings[index]
    sum += var
    hypothesis_embeddings[sent] = var
    index += 1

  centroid = sum/len(text_sentences)

  content_relevance_score = content_relevance(centroid,hypothesis_embeddings)
  sentence_novelty_score = sentence_novelty(content_relevance_score,hypothesis_embeddings)
  phrasa_score = normalization_sen(text_sentences, keyword)
  # position_score = sentence_position(text_sentences)
  return total_score(alpha, beta, gamma, teta, content_relevance_score, sentence_novelty_score, phrasa_score, sentence_pos)

def iter_ext(data, alpha, beta, gamma, teta):
    extractive = []
    index = 0

    result_dict = defaultdict(list)

    model = BertModel.from_pretrained('bert-base-uncased')

    for i in range(len(data)):
        model = BertModel.from_pretrained('bert-base-uncased')

        # embedding = np.empty((len(data[i]), 768))
        embedding = np.empty((len(data[i]), model.config.hidden_size))
        sentence_pos = sentence_position(data[i])

        for e in range (len(data[i])):
          embedding[e,:] = sentence_emb(data[i][e], model)
          # embedding[e,:] = sbert_model.encode(data[i][e])[0]

        # print(len(txt))
        predicted_summary = convert_to_embeddings(alpha, beta, gamma, teta, data[i], embedding, data_key[i], sentence_pos)

        result_dict[i].append(predicted_summary)

    return result_dict

# params = {'alpha': [0.8, 0.7, 0.7, 0.7, 0.7, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4, 0.3],
#          'beta': [0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.3, 0.4, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.3, 0.3],
#          'gamma': [0.05, 0.1, 0.05, 0.15, 0.05, 0.15, 0.10, 0.20, 0.10, 0.15, 0.05, 0.05, 0.05, 0.05, 0.15, 0.15, 0.2, 0.1, 0.2, 0.1, 0.2, 0.1, 0.2],
#          'teta': [0.05, 0.1, 0.15, 0.05, 0.05, 0.15, 0.20, 0.10, 0.10, 0.05, 0.15, 0.05, 0.05, 0.15, 0.05, 0.15, 0.1, 0.2, 0.2, 0.1, 0.1, 0.2, 0.2],}
# scores = []
# for i in tqdm(range(len(params['alpha']))):
#    extract_skp = iter_ext(data, params['alpha'][i], params['beta'][i], params['gamma'][i], params['teta'][i])
#    raw_ext1 = summary2(extract_skp)
#    scores.append(evaluator.get_scores(raw_ext1, doc_abs))
# extract_skp = iter_ext(data)

In [233]:
extract_skp = iter_ext(data, 0.6, 0.2, 0.1, 0.1)

In [241]:
raw_ext1 = summary2(extract_skp)
scores = evaluator.get_scores(raw_ext1, doc_abs)
# scores

In [242]:
rouge_1 = []
rouge_2 = []
rouge_l = []
for i in range(len(raw_ext1)):
    rouge_1.append(scores['rouge-1'][i]['f'][0])
    rouge_2.append(scores['rouge-2'][i]['f'][0])
    rouge_l.append(scores['rouge-l'][i]['f'][0])

experiment = pd.DataFrame()
experiment['ext_sentkey'] = raw_ext1
experiment['referensi'] = doc_abs
experiment['r1'] = rouge_1
experiment['r2'] = rouge_2
experiment['rl'] = rouge_l

In [243]:
experiment.to_csv('./experiment_amazon.csv')

In [179]:
import json
with open("./bobot-amazon2.json","w") as f:
    json.dump(scores,f, default=int)

Position

In [75]:
def total_score(alpha, beta, gamma, content_relevance_score, sentence_novelty_score, position_score):
  final_score = {}
  for sent in content_relevance_score:
    final_score[sent] = alpha*content_relevance_score[sent]+beta*sentence_novelty_score[sent]+gamma*position_score[sent]
  final_score = {k: v for k, v in sorted(final_score.items(), key=lambda item: item[1],reverse=True)}

  return final_score

def convert_to_embeddings(alpha, beta, gamma, text_sentences, text_embeddings, text_position):
  hypothesis_embeddings = {}
  sum = 0
  index = 0

  for sent in text_sentences:
    var = text_embeddings[index]
    sum += var
    hypothesis_embeddings[sent] = var
    index += 1

  centroid = sum/len(text_sentences)

  content_relevance_score = content_relevance(centroid,hypothesis_embeddings)
  sentence_novelty_score = sentence_novelty(content_relevance_score,hypothesis_embeddings)
  position_score = sentence_position(text_position)
  return total_score(alpha, beta, gamma, content_relevance_score, sentence_novelty_score, position_score)

from collections import defaultdict
def iter_ext(data):
    extractive = []
    index = 0

    result_dict = defaultdict(list)

    for i in tqdm(range(len(data))):
        total = 0
        print('data ke ' + str(i))
        model = BertModel.from_pretrained('bert-base-uncased')

        # embedding = np.empty((len(data[i]), 768))
        embedding = np.empty((len(data[i]), model.config.hidden_size))

        for e in tqdm(range (len(data[i]))):
          embedding[e,:] = sentence_emb(data[i][e], model)
          # embedding[e,:] = sbert_model.encode(data[i][e])[0]

        # print(len(txt))
        predicted_summary = convert_to_embeddings(0.6, 0.2, 0.2, data[i], embedding, data[i])

        result_dict[i].append(predicted_summary)

    return result_dict

extract_p = iter_ext(data)

  0%|          | 0/60 [00:00<?, ?it/s]

data ke 0


  2%|▏         | 1/60 [00:06<05:58,  6.08s/it]

data ke 1


  3%|▎         | 2/60 [00:12<06:03,  6.27s/it]

...

 97%|█████████▋| 58/60 [06:02<00:12,  6.28s/it]

data ke 58


 98%|█████████▊| 59/60 [06:08<00:06,  6.28s/it]

data ke 59


100%|██████████| 60/60 [06:14<00:00,  6.25s/it]


In [85]:
raw_ext2 = summary2(extract_p)
scores2 = evaluator.get_scores(raw_ext2, doc_abs)
scores2

{'rouge-1': {'f': 0.3203641359661724,
  'p': 0.2681616605529101,
  'r': 0.4056720495004435},
 'rouge-4': {'f': 0.004705044838213981,
  'p': 0.004002460271382091,
  'r': 0.005894212476856367},
 'rouge-3': {'f': 0.01495100221726089,
  'p': 0.01259920426740929,
  'r': 0.01890468715604596},
 'rouge-2': {'f': 0.057072433871351316,
  'p': 0.0477595298576478,
  'r': 0.07252990594144279},
 'rouge-l': {'f': 0.34485272792798544,
  'p': 0.2960199947711006,
  'r': 0.41853099721625087}}

In [315]:
import bert_score
from bert_score import score

eP, eR, eF1 = score(raw_ext1, doc_abs, model_type="bert-base-uncased", num_layers=12, verbose=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


100%|██████████| 4/4 [00:29<00:00,  7.40s/it]


computing greedy matching.


100%|██████████| 3/3 [00:00<00:00, 17.90it/s]


done in 29.77 seconds, 6.05 sentences/sec


In [316]:
eF1.mean()

tensor(0.6476)

In [313]:
eP, eR, eF1 = score(raw_ext2, doc_abs, model_type="bert-base-uncased", num_layers=12, verbose=True)
eF1.mean()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


100%|██████████| 4/4 [00:23<00:00,  5.98s/it]


computing greedy matching.


100%|██████████| 3/3 [00:00<00:00, 21.18it/s]

done in 24.09 seconds, 7.47 sentences/sec


tensor(0.6493)